## Two simple schedules

In [ ]:
# Begin - startup boilerplate code

import pkgutil

if 'fibertree_bootstrap' not in [pkg.name for pkg in pkgutil.iter_modules()]:
  !python3 -m pip  install git+https://github.com/Fibertree-project/fibertree-bootstrap --quiet

# End - startup boilerplate code


from fibertree_bootstrap import *
fibertree_bootstrap(style="tree", animation='movie', logger=False)


In [ ]:
f = Tensor.fromUncompressed(rank_ids=["V"],
                            root=[ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
displayTensor(f)



In [ ]:
# Set the number of PEs

PEs = 4

## Parallel schedule - v1

Divide the work by the number of PEs then process each chunk in order

In [ ]:
st1 = f.splitEqual(PEs)
displayTensor(st1)

In [ ]:
# Show the parallelism

canvas = createCanvas(f, st1)

for v1_pos, (v1, st1_v0) in enumerate(st1):
    for v0_pos, (v0, _) in enumerate(st1_v0):
        canvas.addActivity((v0,), (v1, v0), spacetime=(v0_pos, v1_pos))
        
displayCanvas(canvas)

## Parallel schedule - v2

Schedule that divides the work and then processes the sequenntially in each fiber. To implement that in the fibertree language we 


In [ ]:
import math

partition_size = math.ceil(len(f.getRoot()) / PEs)

print(f"Partition size: {partition_size}\n")

# Partition (but use relative coordinates, so the swap works)
st2_split = f.splitEqual(partition_size, relativeCoords=True)
print("\n\nSchedule split - note repeated coordinates in lower rank")
displayTensor(st2_split)

# Swap the ranks so the parallel work appears in a single fiber
st2 = st2_split.swapRanks()
print("\n\nSchedule swapped - note again the repeated coordintes in lower rank")
displayTensor(st2)

# Restore to absolute coordinates
for v0, st2_v1 in st2:
    st2_v1.updateCoords(lambda n, c, p: v0 + c)
    
print("\n\nFinal schedule - with original coordinates")
displayTensor(st2)

In [ ]:
# Show the parallelism

canvas = createCanvas(f, st2)

for v0_pos, (v0, st2_v1) in enumerate(st2):
    for v1_pos, (v1, _) in enumerate(st2_v1):
        canvas.addActivity((v1,), (v0, v1), spacetime=(v1_pos, v0_pos))
        
displayCanvas(canvas)